<a href="https://colab.research.google.com/github/rajeshsahu09/CS69002_9A_18CS60R19/blob/master/DL_Assign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movie Review Sentiment Analysis

# Run on **GPU**



### Import Header Files

In [0]:
import torch
import pandas as pd
import numpy as np
import io
import time

### Load the Dataset and Visualise

In [0]:
# from google.colab import files
# uploaded = files.upload()

# import io
# df_train = pd.read_csv(io.StringIO(uploaded['Train_20K.csv'].decode('utf-8')), sep='\t')
# df_train.head()
url = "https://raw.githubusercontent.com/binny-mathew/IITKGP_CS69002_Spring_2019/master/Dataset/Train_20K.csv"
df = pd.read_csv(url, sep='\t')

In [3]:
df.shape

(17999, 2)

In [0]:
df_1 = df[df['label']==1] # get only label == 1
df_0 = df[df['label']==0] # get only label == 0

range_1 = int(0.9*len(df_1))
range_2 = int(0.9*len(df_0))

np.random.seed(42) # fix the seed value
df_1 = df_1.iloc[np.random.permutation(len(df_1))] # shuffled the data set of label==1
df_0 = df_0.iloc[np.random.permutation(len(df_0))] # shuffled the data set of label==0

temp_1_train = df_1.iloc[:range_1]
temp_2_train = df_0.iloc[:range_2]
df_train = pd.concat([temp_1_train, temp_2_train])

temp_1_val = df_1.iloc[range_1:]
temp_2_val = df_0.iloc[range_2:]
df_val = pd.concat([temp_1_val, temp_2_val])

In [5]:
df_train = df_train.iloc[np.random.permutation(len(df_train))] # shuffled the data set of label==1
df_val = df_val.iloc[np.random.permutation(len(df_val))] # shuffled the data set of label==1
len(df_train), len(df_val)

(16198, 1801)

In [6]:
print('Number of Negative movie reviews', len(df_train[df_train['label']==0]))
print('Number of Positive movie reviews', len(df_train[df_train['label']==1]))
print('Number of movie reviews', len(df_train['label']))

Number of Negative movie reviews 8094
Number of Positive movie reviews 8104
Number of movie reviews 16198


In [7]:
print('Number of Negative movie reviews', len(df_val[df_val['label']==0]))
print('Number of Positive movie reviews', len(df_val[df_val['label']==1]))
print('Number of movie reviews', len(df_val['label']))

Number of Negative movie reviews 900
Number of Positive movie reviews 901
Number of movie reviews 1801


In [0]:
url = "https://raw.githubusercontent.com/binny-mathew/IITKGP_CS69002_Spring_2019/master/Dataset/Test_5K.csv"
df_test = pd.read_csv(url, sep='\t')

## Data pre-processing

### Get Train Data

In [0]:
train_raw_text_reviews = df_train['text'].astype(str).tolist()
train_text_labels = df_train['label'].astype(int).tolist()

###Get Validation Data

In [0]:
val_raw_text_reviews = df_val['text'].astype(str).tolist()
val_text_labels = df_val['label'].astype(int).tolist()

###Get Test Data

In [0]:
test_raw_text_reviews = df_test['text'].astype(str).tolist()
test_text_labels = df_test['label'].astype(int).tolist()

#### Cleaning the raw input data

In [12]:
import re
import string
# import nltk
# nltk.download("stopwords")
# from nltk.corpus import stopwords

# # Finding stop words
# stop_words = set(stopwords.words('english'))

import spacy
print('spaCy Version: %s' % (spacy.__version__))
spacy_nlp = spacy.load('en_core_web_sm')
# stop word list
stop_words = set(spacy.lang.en.stop_words.STOP_WORDS)

spaCy Version: 2.0.18


In [0]:
def preprocess_document(doc):
    # negative sense should not be eleminated + some short representation
    CONTRACTIONS = {"didn't":"did not", "mayn't":"may not", "can't":"can not", "won't":"will not", "isn't":"is not", "amn't":"am not",\
                  "aren't":"are not", "wasn't":"was not", "weren't":"were not", "couldn't":"could not", \
                  "wouldn't":"would not", "don't":"do not", "doesn't":"does not",\
                  "i'll":"i will", "you'll":"you will", "they'll":"they will",\
                  "may've":"may have", "can've":"can have", "will've":"will have", "you've":"you have", \
                  "could've":"could have", "would've":"would have", "you've":"you have", "they":"they have",\
                  "i've":"i have", "you've":"you have", "we've":"we have", "there's":"there is", "i'm":"i am",\
                  "it's":"it is", "what's":"what is", "where's":"where is", "how's":"how is", "i'd":"i had"}
    punctuation = string.punctuation + "\n\n"
    punc_replace = ''.join([' ' for s in punctuation]) # required for replacing punctuation with null ('')
    doc_clean = doc.replace('-', ' ') # replace - with null str
    doc_clean = (doc_clean.encode('ascii', 'ignore')).decode("utf-8")
    doc_clean = doc_clean.replace('<br />', '') # replace <br /> with ''
    doc_clean = doc_clean.replace("’", "'") # replace <br /> with null str
    doc_clean = [CONTRACTIONS[word] if word in CONTRACTIONS else word for word in doc_clean.split(' ')] # replacing some common short forms
    doc_clean = " ".join(doc_clean) # list to sentence of strings
    doc_clean = re.sub(r'\W +', ' ', doc_clean) # except [a-zA-Z0-9_]
    doc_clean = re.sub(r'\d+', ' ', doc_clean) # remove numbers [0-9]
    trans_table = str.maketrans(punctuation, punc_replace); # replace punctuations with ' '
    doc_clean = ' '.join([word.translate(trans_table) for word in doc_clean.split(' ')])
    doc_clean = doc_clean.split(' ')
    doc_clean = [word for word in doc_clean if len(word) > 0]
    # removing the stopwords from a sentence
    doc_clean = [word for word in doc_clean if not word.lower() in stop_words or word.lower() == 'not' or word.lower() == 'no']
    return doc_clean

In [14]:
train_text_reviews = [preprocess_document(review.lower()) for review in train_raw_text_reviews]
print (train_text_reviews[len(train_text_reviews)-2])
print (train_text_labels[len(train_text_labels)-2])

['tales', 'told', 'film', 'shot', 'knowledge', 'combined', 'vignette', 'film', 'makers', 'relate', 'vignettes', 'having', 'connected', 'shrink', 'martin', 'kove', 'leads', 'kove', 'vignette', 'sexy', 'vivian', 'schilling', 'woman', 'afraid', 'sun', 'makes', 'adrian', 'monk', 'look', 'brave', 'having', 'paranoia', 'laced', 'evening', 'home', 'literally', 'scream', 'vivian', 'ridiculous', 'things', 'spends', 'majority', 'time', 'nighty', 'shows', 'amazing', 'features', 'film', 'worst', 'not', 'nail', 'biting', 'second', 'vignette', 'owned', 'bill', 'paxton', 'portrays', 'roommate', 'hell', 'geeky', 'roommate', 'allows', 'complete', 'advantage', 'bill', 'vignette', 'funny', 'man', 'fears', 'death', 'moment', 'like', 'pal', 'choked', 'death', 'olive', 'not', 'interesting', 'movie', 'chopped', 'little', 'thought', 'involved', 'bill', 'paxton', 'fans']
0


In [15]:
val_text_reviews = [preprocess_document(review.lower()) for review in val_raw_text_reviews]
print (val_text_reviews[len(val_text_reviews)-2])
print (val_text_labels[len(val_text_labels)-2])

['not', 'realize', 'sellers', 'poor', 'health', 'time', 'filming', 'passed', 'away', 'film', 's', 'release', 'fiendish', 'plot', 'dr', 'fu', 'manchu', 'painful', 'viewing', 'supposedly', 'lampoon', 'sax', 'rohmer', 's', 'famous', 'oriental', 'villain', 'lacks', 'focus', 'potential', 'satirical', 'commentary', 'anti', 'oriental', 'overtones', 'rohmer', 's', 'concept', 'ignored', 'movie', 'employs', 'racist', 'insults', 'hardly', 'actual', 'jokes', 'gags', 'actors', 'behaving', 'idiotically', 'spouting', 'dreary', 'lines', 'especially', 'distressing', 'sid', 'caesar', 'forced', 'spout', 'curses', 'racial', 'slurs', 'attempted', 'laughs', 'actors', 'embarrass', 'peter', 'sellers', 'plays', 'dual', 'roles', 'sinister', 'fu', 'manchu', 'trying', 'concoct', 'formula', 'regain', 'youth', 'stalwart', 'british', 'foe', 'nayland', 'smith', 'sellers', 'not', 'cent', 'bad', 'conveys', 'quirky', 'warmth', 'smith', 'discusses', 'fetishistic', 'attachment', 'lawn', 'mower', 's', 'oddly', 'moving', 'm

In [16]:
test_text_reviews = [preprocess_document(review.lower()) for review in test_raw_text_reviews]
print (test_text_reviews[len(test_text_reviews)-2])
print (test_text_labels[len(test_text_labels)-2])

['shining', 'know', 'weird', 'movie', 'movie', 'people', 'claim', 'not', 'like', 'horror', 'films', 'shining', 'terrific', 'film', 'stanley', 'kubrick', 's', 'classic', 'vision', 'stephen', 'king', 's', 'horror', 'tale', 'madness', 'blood', 'incredible', 'film', 'wither', 'seen', 'not', 'heard', 'know', 'lines', 'know', 'classic', 'images', 'forget', 'jack', 's', 's', 'johnny', 'forget', 'work', 'no', 'play', 'jack', 'dull', 'boy', 'forget', 'chilling', 'ending', 'film', 'unforgettable', 'honestly', 'opinion', 'kubrick', 's', 'best', 'work', 'know', 'lot', 'argument', 'department', 'lot', 'people', 'space', 'odyssey', 'clockwork', 'orange', 'dr', 'strangelove', 'film', 'pioneered', 'film', 'making', 'shining', 'perfected', 'tale', 'isolation', 'madness', 'terrifying', 'images', 'ultimate', 'ghost', 'story', 'crawl', 'underneath', 'skin', 'jack', 'torrance', 'jack', 's', 'son', 'danny', 'jack', 's', 'wife', 'wendy', 'arrive', 'overlook', 'hotel', 'closing', 'day', 'elderly', 'african', 

In [17]:
import gc
gc.collect()

0

In [0]:
def build_vocabulary(sentences):
    # Build vocabulary
    dictWordCount = {}
    for sent in sentences:
        for word in sent:
            dictWordCount[word] = 0 # initialising the dict value to zero
    for sent in sentences:
        for word in sent:
            dictWordCount[word] += 1 # updating the dictionary count
    
    temp = dictWordCount.copy()
    for key, val in temp.items():
        if(dictWordCount[key] <= 10):
            del dictWordCount[key]
    
    # Mapping from index to word
    vocabulary_inv = sorted(dictWordCount, key=dictWordCount.__getitem__, reverse=True)
    
    # Mapping from word to index
    vocabulary = {x: i for i, x in enumerate(vocabulary_inv)}
    return vocabulary, vocabulary_inv

###Creating Tokens and Types

In [0]:
word_to_ix, ix_to_word = build_vocabulary(train_text_reviews+val_text_reviews+test_text_reviews)

In [20]:
VOCAB_SIZE = len(word_to_ix)
ix_to_word[word_to_ix['kick']]=='kick', word_to_ix['kick'], VOCAB_SIZE

(True, 1710, 17746)

In [21]:
gc.collect()

403

## PyTorch

In [0]:
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable # Automatic gradients are calculated and back-propagated through the computational graph
import copy
import csv
import time
from google.colab import files

## Generate the BOW Vectors

In [0]:
def make_bow_vector(sentence, word_to_ix):
    # create a vector of zeros of vocab size = len(word_to_idx)
    vec = torch.zeros(len(word_to_ix)).to('cuda:0') #, device=device) # make 1D vector of len = vocab size
    for word in sentence:
        if word not in word_to_ix:            
#             raise ValueError('Word',word,' not present in the dictionary. Sorry!')
            pass
        else:
            vec[word_to_ix[word]]+=1 # count the number of occurance of same word in a sentences
            
    return vec.view(1, -1)

### Prepare Input Train and Validation torch vector

In [24]:
# store the bag of word vectors for each sentences and wraping to tensor of torch type
tic = time.time()
train_data = [Variable(make_bow_vector(instance, word_to_ix)).to('cuda:0') for instance in train_text_reviews]
num_train_data = len(train_text_reviews)

val_data = [Variable(make_bow_vector(instance, word_to_ix)).to('cuda:0') for instance in val_text_reviews]
num_val_data = len(val_text_reviews)
toc = time.time()
num_train_data, num_val_data, (toc-tic)

(16198, 1801, 98.2123191356659)

### Prepare Test Instance

In [0]:
data_test = []
data_test = [Variable(make_bow_vector(instance, word_to_ix)).to('cuda:0') for instance in test_text_reviews]

### Validation Accuracy Computation

In [0]:
def evaluate_validation_accuracy(data, net):
    sum_loss = 0
    
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    
    for i, instance in enumerate(data):
        label = val_text_labels[i] # get the label of the corresponding instace
        label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label
        
#         vec = Variable(make_bow_vector(instance, word_to_ix)).to('cuda:0') # wrap to tensor of torch type for instance
        
#         prob = net.forward(vec) # forward pass

        prob = net.forward(instance)
    
        _class = 1 if prob.item() > 0.5 else 0 # sigmoid activated
        
        loss = loss_function(prob, label) # compute the loss
        
        sum_loss += float(loss.item())
        
#         if(int(label) == _class and _class == 1):
#             true_positive += 1
            
#         if(int(label) == _class and _class == 0):
#             true_negative += 1
        
#         if(_class == 1 and int(label) == 0):
#             false_positive += 1
            
#         if(_class == 0 and int(label) == 1):
#             false_negative += 1
    
    
#     precision = float(true_positive) / (true_positive + false_positive)
#     recall = float(true_positive) / (true_positive + false_negative)
#     f_score = float(2)*precision*recall / (precision + recall)
#     accuracy = float(1)*(true_positive+true_negative)/(true_positive+true_negative+false_positive+false_negative)
    
    return float(sum_loss)/len(data)#, float(100)*accuracy, precision, recall, f_score

##Task-1

In [0]:
task1 = [(50, 'Task1A.mdl'), (100, 'Task1B.mdl'), (200, 'Task1C.mdl'), (500, 'Task1D.mdl')]

In [0]:
for num_of_hidden, task_name in task1:
    class BOWClassifier(nn.Module):
        def __init__(self, input_size, hidden_size, output_size):
            super(BOWClassifier, self).__init__()
            SEED = 42
            torch.manual_seed(SEED)
            torch.cuda.manual_seed(SEED)
            self.hidden_size = hidden_size
            self.i2h = nn.Linear(input_size, hidden_size) # initialises weights and biases i2h
            self.h2o = nn.Linear(hidden_size, output_size) # initialises weights and biases h2o

        def forward(self, x):
            x = torch.relu(self.i2h(x)) # relu activation @ hidden layer
            x = torch.sigmoid(self.h2o(x)) # sigmoid activation @ output layer
            return x

    num_of_input = VOCAB_SIZE # Bag word restriction has to be equal to vocabulary
    num_of_output = 1 # binary sentiment classes (+ve, -ve)

    bow = BOWClassifier(num_of_input, num_of_hidden, num_of_output).to('cuda:0') # initialises weights and biases
    
    # define a loss function and an optimizer
    loss_function = nn.BCELoss()
    opt = torch.optim.SGD(bow.parameters(), lr = 0.0001)
    
    gc.collect()

    # Train The Model
    epochs = 100
    # the training loop
    total_time = 0.0
    prev_val_loss = float('inf')
    val_loss = 0
    early_stop_bow = 0

    for e in range(epochs):
        tic = time.time() # start the timer
        correct = 0
        cumulative_loss = 0
        incorrect = 0

        for i, instance in enumerate(train_data): # train_text_reviews 
            # get the training data
            label = train_text_labels[i] # get the label of the corresponding instace
            label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label

            bow.zero_grad() # reset the gradient for each instance

            pred = bow.forward(instance)

            loss = loss_function(pred, label) # compute the loss
            loss.backward() # backprop the loss
            opt.step() # performs parameter updation based on the current gradient

            cumulative_loss += float(loss.item()) # accumulate the loss over whole training sample

            pred_class = 1 if pred.item() > 0.5 else 0 # as sigmoid activated
            if(int(label) == pred_class): # counting correct prediction in each epoch
                correct += 1

        train_loss = float(cumulative_loss)/num_train_data
        train_accuracy = correct*float(100)/num_train_data

#         val_loss, val_accuracy, p, r, f = evaluate_validation_accuracy(val_data, bow) # test val-data-set on currently trained model
        val_loss = evaluate_validation_accuracy(val_data, bow) # test val-data-set on currently trained model
        
        if(prev_val_loss-val_loss < 0.0): # early stop if nearly no change
            bow = copy.deepcopy(early_stop_bow)
            break

        early_stop_bow = copy.deepcopy(bow)
        
        prev_val_loss = val_loss

        toc = time.time() # final time
        total_time += (toc-tic)

        print("Epoch {}/{}\n[On Training] ==> Time: {:.2f}s, Train Loss: {:.9f}, Train Accuracy: {:.2f}%".format(e+1, epochs, (toc-tic), train_loss, train_accuracy))
#         print("[On Validation] ==> Precision: {:.3f}, Recall: {:.3f}, F-Score: {:.3f}, Val loss: {:.9f}, Val Accuracy: {:.2f}% Total Time: {:.2f}s".format(p, r, f, val_loss, val_accuracy, total_time))
        print("[On Validation] ==> Val loss: {:.9f}, Total Time: {:.2f}s".format(val_loss, total_time))
    
    torch.save(bow, task_name) ## Save the model
    
    print('--- AFTER TRAINING ---\tModel: {}'.format(task_name))
    correct = 0
    tic = time.time()
    for i, instance in enumerate(data_test):
        label = test_text_labels[i] # get the label of the corresponding instace
        label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label

        pred = bow.forward(instance)
        pred_class = 1 if pred.item() > 0.5 else 0 # sigmoid activated

        if(int(label) == pred_class):
            correct += 1
    toc = time.time()
    print("[On Testing] Time: {}, Test Accuracy: {:.2f}%".format((toc-tic), correct*100/len(data_test)))
    
    print ('\n')

In [0]:
files.download('Task1A.mdl')
files.download('Task1B.mdl')
files.download('Task1C.mdl')
files.download('Task1D.mdl')

##Task-2

In [0]:
task2 = [(10, 10, 'Task2A.mdl'), (20, 10, 'Task2B.mdl'), (30, 30, 'Task2C.mdl'), (50, 50, 'Task2D.mdl'), (100, 50, 'Task2E.mdl')]

In [0]:
for num_of_hidden1, num_of_hidden2, task_name in task2:
    class BOWClassifier(nn.Module):
        def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
            super(BOWClassifier, self).__init__()
            SEED = 42
            torch.manual_seed(SEED)
            torch.cuda.manual_seed(SEED)
            self.i2h = nn.Linear(input_size, hidden_size1) # initialises weights and biases i2h
            self.h2h = nn.Linear(hidden_size1, hidden_size2) # initialises weights and biases h2h
            self.h2o = nn.Linear(hidden_size2, output_size) # initialises weights and biases h2o

        def forward(self, x):
            x = torch.relu(self.i2h(x)) # relu activation @ hidden layer
            x = torch.relu(self.h2h(x)) # relu activation @ hidden layer
            x = torch.sigmoid(self.h2o(x)) # sigmoid activation @ output layer
            return x

    num_of_input = VOCAB_SIZE # Bag word restriction has to be equal to vocabulary
    num_of_output = 1 # binary sentiment classes (+ve, -ve)

    bow = BOWClassifier(num_of_input, num_of_hidden1, num_of_hidden2, num_of_output).to('cuda:0') # initialises weights and biases

    # define a loss function and an optimizer
    loss_function = nn.BCELoss()
    opt = torch.optim.SGD(bow.parameters(), lr = 0.0001)

    gc.collect()

    # Train The Model
    epochs = 100
    # the training loop
    total_time = 0.0
    prev_val_loss = float('inf')
    val_loss = 0
    early_stop_bow = 0

    for e in range(epochs):
        tic = time.time() # start the timer
        correct = 0
        cumulative_loss = 0
        incorrect = 0

        for i, instance in enumerate(train_data): # train_text_reviews 
            # get the training data
            label = train_text_labels[i] # get the label of the corresponding instace
            label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label

            bow.zero_grad() # reset the gradient for each instance

            pred = bow.forward(instance)

            loss = loss_function(pred, label) # compute the loss
            loss.backward() # backprop the loss
            opt.step() # performs parameter updation based on the current gradient

            cumulative_loss += float(loss.item()) # accumulate the loss over whole training sample

            pred_class = 1 if pred.item() > 0.5 else 0 # as sigmoid activated
            if(int(label) == pred_class): # counting correct prediction in each epoch
                correct += 1

        train_loss = float(cumulative_loss)/num_train_data
        train_accuracy = correct*float(100)/num_train_data

#         val_loss, val_accuracy, p, r, f = evaluate_validation_accuracy(val_data, bow) # test val-data-set on currently trained model
        val_loss = evaluate_validation_accuracy(val_data, bow) # test val-data-set on currently trained model
        
        if(prev_val_loss-val_loss < 0.0): # early stop if nearly no change
            bow = copy.deepcopy(early_stop_bow)
            break

        early_stop_bow = copy.deepcopy(bow)

        prev_val_loss = val_loss

        toc = time.time() # final time
        total_time += (toc-tic)

        print("Epoch {}/{}\n[On Training] ==> Time: {:.2f}s, Train Loss: {:.9f}, Train Accuracy: {:.2f}%".format(e+1, epochs, (toc-tic), train_loss, train_accuracy))
#         print("[On Validation] ==> Precision: {:.3f}, Recall: {:.3f}, F-Score: {:.3f}, Val loss: {:.9f}, Val Accuracy: {:.2f}% Total Time: {:.2f}s".format(p, r, f, val_loss, val_accuracy, total_time))
        print("[On Validation] ==> Val loss: {:.9f}, Total Time: {:.2f}s".format(val_loss, total_time))

    torch.save(bow, task_name)

    print('--- AFTER TRAINING ---\tModel: {}'.format(task_name))
    correct = 0
    tic = time.time()
    for i, instance in enumerate(data_test):
        label = test_text_labels[i] # get the label of the corresponding instace
        label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label

        pred = bow.forward(instance)
        pred_class = 1 if pred.item() > 0.5 else 0 # sigmoid activated

        if(int(label) == pred_class):
            correct += 1
    toc = time.time()
    print("[On Testing] Time: {}, Test Accuracy: {:.2f}%".format((toc-tic), correct*100/len(data_test)))
    
    print("\n")

In [0]:
files.download('Task2A.mdl')
files.download('Task2B.mdl')
files.download('Task2C.mdl')
files.download('Task2D.mdl')
files.download('Task2E.mdl')

##Task-3

In [0]:
task3 = [(100, 50, 10, 'Task3A.mdl'), (200, 100, 10, 'Task3B.mdl')]

In [0]:
for num_of_hidden1, num_of_hidden2, num_of_hidden3, task_name in task3:
    class BOWClassifier(nn.Module):
        def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
            super(BOWClassifier, self).__init__()
            SEED = 42
            torch.manual_seed(SEED)
            torch.cuda.manual_seed(SEED)
            self.i2h1 = nn.Linear(input_size, hidden_size1) # initialises weights and biases i2h1
            self.h12h2 = nn.Linear(hidden_size1, hidden_size2) # initialises weights and biases h12h2
            self.h22h3 = nn.Linear(hidden_size2, hidden_size3) # initialises weights and biases h22h3
            self.h32o = nn.Linear(hidden_size3, output_size) # initialises weights and biases h32o

        def forward(self, x):
            x = torch.relu(self.i2h1(x)) # relu activation @ hidden layer
            x = torch.relu(self.h12h2(x)) # relu activation @ hidden layer
            x = torch.relu(self.h22h3(x)) # relu activation @ hidden layer
            x = torch.sigmoid(self.h32o(x)) # sigmoid activation @ output layer
            return x

    num_of_input = VOCAB_SIZE # Bag word restriction has to be equal to vocabulary
    num_of_output = 1 # binary sentiment classes (+ve, -ve)

    bow = BOWClassifier(num_of_input, num_of_hidden1, num_of_hidden2, num_of_hidden3, num_of_output).to('cuda:0') # initialises weights and biases

    # define a loss function and an optimizer
    loss_function = nn.BCELoss()
    opt = torch.optim.SGD(bow.parameters(), lr = 0.0001)

    gc.collect()

    # Train The Model
    epochs = 100
    # the training loop
    total_time = 0.0
    prev_val_loss = float('inf')
    val_loss = 0
    early_stop_bow = 0

    for e in range(epochs):
        tic = time.time() # start the timer
        correct = 0
        cumulative_loss = 0
        incorrect = 0

        for i, instance in enumerate(train_data): # train_text_reviews 
            # get the training data
            label = train_text_labels[i] # get the label of the corresponding instace
            label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label

            bow.zero_grad() # reset the gradient for each instance

            pred = bow.forward(instance)

            loss = loss_function(pred, label) # compute the loss
            loss.backward() # backprop the loss
            opt.step() # performs parameter updation based on the current gradient

            cumulative_loss += float(loss.item()) # accumulate the loss over whole training sample

            pred_class = 1 if pred.item() > 0.5 else 0 # as sigmoid activated
            if(int(label) == pred_class): # counting correct prediction in each epoch
                correct += 1

        train_loss = float(cumulative_loss)/num_train_data
        train_accuracy = correct*float(100)/num_train_data

#         val_loss, val_accuracy, p, r, f = evaluate_validation_accuracy(val_data, bow) # test val-data-set on currently trained model
        val_loss = evaluate_validation_accuracy(val_data, bow) # test val-data-set on currently trained model

        if(prev_val_loss-val_loss < 0.0): # early stop if nearly no change
            bow = copy.deepcopy(early_stop_bow)
            break

        early_stop_bow = copy.deepcopy(bow)

        prev_val_loss = val_loss

        toc = time.time() # final time
        total_time += (toc-tic)

        print("Epoch {}/{}\n[On Training] ==> Time: {:.2f}s, Train Loss: {:.9f}, Train Accuracy: {:.2f}%".format(e+1, epochs, (toc-tic), train_loss, train_accuracy))
#         print("[On Validation] ==> Precision: {:.3f}, Recall: {:.3f}, F-Score: {:.3f}, Val loss: {:.9f}, Val Accuracy: {:.2f}% Total Time: {:.2f}s".format(p, r, f, val_loss, val_accuracy, total_time))
        print("[On Validation] ==> Val loss: {:.9f}, Total Time: {:.2f}s".format(val_loss, total_time))
    
    torch.save(bow, task_name)
    
    print('--- AFTER TRAINING ---\tModel: {}'.format(task_name))
    correct = 0
    tic = time.time()
    for i, instance in enumerate(data_test):
        label = test_text_labels[i] # get the label of the corresponding instace
        label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label

        pred = bow.forward(instance)
        pred_class = 1 if pred.item() > 0.5 else 0 # sigmoid activated

        if(int(label) == pred_class):
            correct += 1
    toc = time.time()
    print("[On Testing] Time: {}, Test Accuracy: {:.2f}%".format((toc-tic), correct*100/len(data_test)))

    print("\n")

In [0]:
files.download('Task3A.mdl')
files.download('Task3B.mdl')

## TASK-4

In [0]:
task4 = [(30, 20, 10, 'Task4A.mdl'), (100, 100, 0, 'Task4B.mdl'), (100, 10, 0, 'Task4C.mdl')]

In [0]:
k = 1
for num_of_hidden1, num_of_hidden2, num_of_hidden3, task_name in task4:
    if(k == 1):
        k = 2
        class BOWClassifier(nn.Module):
            def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
                super(BOWClassifier, self).__init__()
                SEED = 42
                torch.manual_seed(SEED)
                torch.cuda.manual_seed(SEED)
                self.i2h1 = nn.Linear(input_size, hidden_size1) # initialises weights and biases i2h1
                self.drop1 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h12h2 = nn.Linear(hidden_size1, hidden_size2) # initialises weights and biases h12h2
                self.drop2 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h22h3 = nn.Linear(hidden_size2, hidden_size3) # initialises weights and biases h22h3
                self.drop3 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h32o = nn.Linear(hidden_size3, output_size) # initialises weights and biases h32o

            def forward(self, x):
                x = torch.relu(self.i2h1(x)) # relu activation @ hidden layer
                x = torch.relu(self.h12h2(x)) # relu activation @ hidden layer
                x = torch.relu(self.h22h3(x)) # relu activation @ hidden layer
                x = torch.sigmoid(self.h32o(x)) # sigmoid activation @ output layer
                return x

        num_of_input = VOCAB_SIZE # Bag word restriction has to be equal to vocabulary
        num_of_output = 1 # binary sentiment classes (+ve, -ve)

        bow = BOWClassifier(num_of_input, num_of_hidden1, num_of_hidden2, num_of_hidden3, num_of_output).to('cuda:0') # initialises weights and biases
    elif(k == 2):
        k = 2
        class BOWClassifier(nn.Module):
            def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
                super(BOWClassifier, self).__init__()
                SEED = 42
                torch.manual_seed(SEED)
                torch.cuda.manual_seed(SEED)
                self.i2h = nn.Linear(input_size, hidden_size1) # initialises weights and biases i2h
                self.drop1 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h2h = nn.Linear(hidden_size1, hidden_size2) # initialises weights and biases i2h
                self.drop2 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h2o = nn.Linear(hidden_size2, output_size) # initialises weights and biases h2o

            def forward(self, x):
                x = torch.relu(self.i2h(x)) # relu activation @ hidden layer
                x = torch.relu(self.h2h(x)) # relu activation @ hidden layer
                x = torch.sigmoid(self.h2o(x)) # sigmoid activation @ output layer
                return x

        num_of_input = VOCAB_SIZE # Bag word restriction has to be equal to vocabulary
        num_of_output = 1 # binary sentiment classes (+ve, -ve)

        bow = BOWClassifier(num_of_input, num_of_hidden1, num_of_hidden2, num_of_output).to('cuda:0') # initialises weights and biases

    # define a loss function and an optimizer
    loss_function = nn.BCELoss()
    opt = torch.optim.SGD(bow.parameters(), lr = 0.0001)

    gc.collect()

    # Train The Model
    epochs = 100
    # the training loop
    total_time = 0.0
    prev_val_loss = float('inf')
    val_loss = 0
    early_stop_bow = 0

    for e in range(epochs):
        tic = time.time() # start the timer
        correct = 0
        cumulative_loss = 0
        incorrect = 0

        for i, instance in enumerate(train_data): # train_text_reviews 
            # get the training data
            label = train_text_labels[i] # get the label of the corresponding instace
            label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label

            bow.zero_grad() # reset the gradient for each instance

            pred = bow.forward(instance)

            loss = loss_function(pred, label) # compute the loss
            loss.backward() # backprop the loss
            opt.step() # performs parameter updation based on the current gradient

            cumulative_loss += float(loss.item()) # accumulate the loss over whole training sample

            pred_class = 1 if pred.item() > 0.5 else 0 # as sigmoid activated
            if(int(label) == pred_class): # counting correct prediction in each epoch
                correct += 1

        train_loss = float(cumulative_loss)/num_train_data
        train_accuracy = correct*float(100)/num_train_data

#         val_loss, val_accuracy, p, r, f = evaluate_validation_accuracy(val_data, bow) # test val-data-set on currently trained model
        val_loss = evaluate_validation_accuracy(val_data, bow) # test val-data-set on currently trained model
    
        if(prev_val_loss-val_loss < 0.0): # early stop if nearly no change
            bow = copy.deepcopy(early_stop_bow)
            break

        early_stop_bow = copy.deepcopy(bow)

        prev_val_loss = val_loss

        toc = time.time() # final time
        total_time += (toc-tic)

        print("Epoch {}/{}\n[On Training] ==> Time: {:.2f}s, Train Loss: {:.9f}, Train Accuracy: {:.2f}%".format(e+1, epochs, (toc-tic), train_loss, train_accuracy))
#         print("[On Validation] ==> Precision: {:.3f}, Recall: {:.3f}, F-Score: {:.3f}, Val loss: {:.9f}, Val Accuracy: {:.2f}% Total Time: {:.2f}s".format(p, r, f, val_loss, val_accuracy, total_time))
        print("[On Validation] ==> Val loss: {:.9f}, Total Time: {:.2f}s".format(val_loss, total_time))

    torch.save(bow, task_name)
    
    print('--- AFTER TRAINING ---\tModel: {}'.format(task_name))
    correct = 0
    tic = time.time()
    for i, instance in enumerate(data_test):
        label = test_text_labels[i] # get the label of the corresponding instace
        label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label

        pred = bow.forward(instance)
        pred_class = 1 if pred.item() > 0.5 else 0 # sigmoid activated

        if(int(label) == pred_class):
            correct += 1
    toc = time.time()
    print("[On Testing] Time: {}, Test Accuracy: {:.2f}%".format((toc-tic), correct*100/len(data_test)))

    print("\n")

In [0]:
files.download('Task4A.mdl')
files.download('Task4B.mdl')
files.download('Task4C.mdl')

## Task-5

In [0]:
task5 = [('relu', 'Task5A.mdl'), ('tanh', 'Task5B.mdl'), ('sigmoid', 'Task5C.mdl')]

In [0]:
for activation, task_name in task5:
    if(activation == 'relu'):
        class BOWClassifier(nn.Module):
            def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
                super(BOWClassifier, self).__init__()
                SEED = 42
                torch.manual_seed(SEED)
                torch.cuda.manual_seed(SEED)
                self.i2h1 = nn.Linear(input_size, hidden_size1) # initialises weights and biases i2h1
                self.drop1 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h12h2 = nn.Linear(hidden_size1, hidden_size2) # initialises weights and biases h12h2
                self.drop2 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h22h3 = nn.Linear(hidden_size2, hidden_size3) # initialises weights and biases h22h3
                self.drop3 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h32o = nn.Linear(hidden_size3, output_size) # initialises weights and biases h32o

            def forward(self, x):
                x = torch.relu(self.i2h1(x)) # relu activation @ hidden layer
                x = torch.relu(self.h12h2(x)) # relu activation @ hidden layer
                x = torch.relu(self.h22h3(x)) # relu activation @ hidden layer
                x = torch.sigmoid(self.h32o(x)) # sigmoid activation @ output layer
                return x

        num_of_input = VOCAB_SIZE # Bag word restriction has to be equal to vocabulary
        num_of_hidden1 = 30
        num_of_hidden2 = 20
        num_of_hidden3 = 10
        num_of_output = 1 # binary sentiment classes (+ve, -ve)

        bow = BOWClassifier(num_of_input, num_of_hidden1, num_of_hidden2, num_of_hidden3, num_of_output).to('cuda:0') # initialises weights and biases
        
    if(activation == 'tanh'):
        class BOWClassifier(nn.Module):
            def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
                super(BOWClassifier, self).__init__()
                SEED = 42
                torch.manual_seed(SEED)
                torch.cuda.manual_seed(SEED)
                self.i2h1 = nn.Linear(input_size, hidden_size1) # initialises weights and biases i2h1
                self.drop1 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h12h2 = nn.Linear(hidden_size1, hidden_size2) # initialises weights and biases h12h2
                self.drop2 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h22h3 = nn.Linear(hidden_size2, hidden_size3) # initialises weights and biases h22h3
                self.drop3 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h32o = nn.Linear(hidden_size3, output_size) # initialises weights and biases h32o

            def forward(self, x):
                x = torch.tanh(self.i2h1(x)) # tanh activation @ hidden layer
                x = torch.tanh(self.h12h2(x)) # tanh activation @ hidden layer
                x = torch.tanh(self.h22h3(x)) # tanh activation @ hidden layer
                x = torch.sigmoid(self.h32o(x)) # sigmoid activation @ output layer
                return x

        num_of_input = VOCAB_SIZE # Bag word restriction has to be equal to vocabulary
        num_of_hidden1 = 30
        num_of_hidden2 = 20
        num_of_hidden3 = 10
        num_of_output = 1 # binary sentiment classes (+ve, -ve)

        bow = BOWClassifier(num_of_input, num_of_hidden1, num_of_hidden2, num_of_hidden3, num_of_output).to('cuda:0') # initialises weights and biases
        
    if(activation == 'sigmoid'):
        class BOWClassifier(nn.Module):
            def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
                super(BOWClassifier, self).__init__()
                SEED = 42
                torch.manual_seed(SEED)
                torch.cuda.manual_seed(SEED)
                self.i2h1 = nn.Linear(input_size, hidden_size1) # initialises weights and biases i2h1
                self.drop1 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h12h2 = nn.Linear(hidden_size1, hidden_size2) # initialises weights and biases h12h2
                self.drop2 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h22h3 = nn.Linear(hidden_size2, hidden_size3) # initialises weights and biases h22h3
                self.drop3 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h32o = nn.Linear(hidden_size3, output_size) # initialises weights and biases h32o

            def forward(self, x):
                x = torch.sigmoid(self.i2h1(x)) # sigmoid activation @ hidden layer
                x = torch.sigmoid(self.h12h2(x)) # sigmoid activation @ hidden layer
                x = torch.sigmoid(self.h22h3(x)) # sigmoid activation @ hidden layer
                x = torch.sigmoid(self.h32o(x)) # sigmoid activation @ output layer
                return x

        num_of_input = VOCAB_SIZE # Bag word restriction has to be equal to vocabulary
        num_of_hidden1 = 30
        num_of_hidden2 = 20
        num_of_hidden3 = 10
        num_of_output = 1 # binary sentiment classes (+ve, -ve)

        bow = BOWClassifier(num_of_input, num_of_hidden1, num_of_hidden2, num_of_hidden3, num_of_output).to('cuda:0') # initialises weights and biases
        
    # define a loss function and an optimizer
    loss_function = nn.BCELoss()
    if(activation == 'sigmoid'):
        opt = torch.optim.SGD(bow.parameters(), lr = 0.001) ## sigmoid activation converges slowly
    else:
        opt = torch.optim.SGD(bow.parameters(), lr = 0.0001)

    gc.collect()

    # Train The Model
    epochs = 100
    # the training loop
    total_time = 0.0
    prev_val_loss = float('inf')
    val_loss = 0
    early_stop_bow = 0

    for e in range(epochs):
        tic = time.time() # start the timer
        correct = 0
        cumulative_loss = 0
        incorrect = 0

        for i, instance in enumerate(train_data): # train_text_reviews 
            # get the training data
            label = train_text_labels[i] # get the label of the corresponding instace
            label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label

            bow.zero_grad() # reset the gradient for each instance

            pred = bow.forward(instance)

            loss = loss_function(pred, label) # compute the loss
            loss.backward() # backprop the loss
            opt.step() # performs parameter updation based on the current gradient

            cumulative_loss += float(loss.item()) # accumulate the loss over whole training sample

            pred_class = 1 if pred.item() > 0.5 else 0 # as sigmoid activated
            if(int(label) == pred_class): # counting correct prediction in each epoch
                correct += 1

        train_loss = float(cumulative_loss)/num_train_data
        train_accuracy = correct*float(100)/num_train_data

#         val_loss, val_accuracy, p, r, f = evaluate_validation_accuracy(val_data, bow) # test val-data-set on currently trained model
        val_loss = evaluate_validation_accuracy(val_data, bow) # test val-data-set on currently trained model

        if(prev_val_loss-val_loss < 0.0): # early stop if nearly no change
            bow = copy.deepcopy(early_stop_bow)
            break

        early_stop_bow = copy.deepcopy(bow)

        prev_val_loss = val_loss

        toc = time.time() # final time
        total_time += (toc-tic)

        print("Epoch {}/{}\n[On Training] ==> Time: {:.2f}s, Train Loss: {:.9f}, Train Accuracy: {:.2f}%".format(e+1, epochs, (toc-tic), train_loss, train_accuracy))
#         print("[On Validation] ==> Precision: {:.3f}, Recall: {:.3f}, F-Score: {:.3f}, Val loss: {:.9f}, Val Accuracy: {:.2f}% Total Time: {:.2f}s".format(p, r, f, val_loss, val_accuracy, total_time))
        print("[On Validation] ==> Val loss: {:.9f}, Total Time: {:.2f}s".format(val_loss, total_time))

    torch.save(bow, task_name)
    
    print('--- AFTER TRAINING ---\tModel: {}'.format(task_name))
    correct = 0
    tic = time.time()
    for i, instance in enumerate(data_test):
        label = test_text_labels[i] # get the label of the corresponding instace
        label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label

        pred = bow.forward(instance)
        pred_class = 1 if pred.item() > 0.5 else 0 # sigmoid activated

        if(int(label) == pred_class):
            correct += 1
    toc = time.time()
    print("[On Testing] Time: {}, Test Accuracy: {:.2f}%".format((toc-tic), correct*100/len(data_test)))

    print("\n")

##Task-6

##Task-7

In [0]:
task7 = [('ce', 'Task7A.mdl'), ('mse', 'Task7B.mdl'), ('hinge', 'Task7C.mdl')]

In [51]:
for loss_fun, task_name in task7:
    if(loss_fun == 'ce'):
        class BOWClassifier(nn.Module):
            def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
                super(BOWClassifier, self).__init__()
                SEED = 42
                torch.manual_seed(SEED)
                torch.cuda.manual_seed(SEED)
                self.i2h1 = nn.Linear(input_size, hidden_size1) # initialises weights and biases i2h1
                self.drop1 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h12h2 = nn.Linear(hidden_size1, hidden_size2) # initialises weights and biases h12h2
                self.drop2 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h22h3 = nn.Linear(hidden_size2, hidden_size3) # initialises weights and biases h22h3
                self.drop3 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h32o = nn.Linear(hidden_size3, output_size) # initialises weights and biases h32o

            def forward(self, x):
                x = torch.relu(self.i2h1(x)) # relu activation @ hidden layer
                x = torch.relu(self.h12h2(x)) # relu activation @ hidden layer
                x = torch.relu(self.h22h3(x)) # relu activation @ hidden layer
                x = torch.sigmoid(self.h32o(x)) # sigmoid activation @ output layer
                return x

        num_of_input = VOCAB_SIZE # Bag word restriction has to be equal to vocabulary
        num_of_hidden1 = 30
        num_of_hidden2 = 20
        num_of_hidden3 = 10
        num_of_output = 1 # binary sentiment classes (+ve, -ve)

        bow = BOWClassifier(num_of_input, num_of_hidden1, num_of_hidden2, num_of_hidden3, num_of_output).to('cuda:0') # initialises weights and biases
        
        # define a loss function and 
        loss_function = nn.BCELoss()
        
    if(loss_fun == 'mse'):
        class BOWClassifier(nn.Module):
            def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
                super(BOWClassifier, self).__init__()
                SEED = 42
                torch.manual_seed(SEED)
                torch.cuda.manual_seed(SEED)
                self.i2h1 = nn.Linear(input_size, hidden_size1) # initialises weights and biases i2h1
                self.drop1 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h12h2 = nn.Linear(hidden_size1, hidden_size2) # initialises weights and biases h12h2
                self.drop2 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h22h3 = nn.Linear(hidden_size2, hidden_size3) # initialises weights and biases h22h3
                self.drop3 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h32o = nn.Linear(hidden_size3, output_size) # initialises weights and biases h32o

            def forward(self, x):
                x = torch.relu(self.i2h1(x)) # relu activation @ hidden layer
                x = torch.relu(self.h12h2(x)) # relu activation @ hidden layer
                x = torch.relu(self.h22h3(x)) # relu activation @ hidden layer
                x = torch.sigmoid(self.h32o(x)) # sigmoid activation @ output layer
                return x

        num_of_input = VOCAB_SIZE # Bag word restriction has to be equal to vocabulary
        num_of_hidden1 = 30
        num_of_hidden2 = 20
        num_of_hidden3 = 10
        num_of_output = 1 # binary sentiment classes (+ve, -ve)

        bow = BOWClassifier(num_of_input, num_of_hidden1, num_of_hidden2, num_of_hidden3, num_of_output).to('cuda:0') # initialises weights and biases
        
        # define a loss function and 
        loss_function = nn.MSELoss()
        
    if(loss_fun == 'hinge'):
        class BOWClassifier(nn.Module):
            def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
                super(BOWClassifier, self).__init__()
                SEED = 42
                torch.manual_seed(SEED)
                torch.cuda.manual_seed(SEED)
                self.i2h1 = nn.Linear(input_size, hidden_size1) # initialises weights and biases i2h1
                self.drop1 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h12h2 = nn.Linear(hidden_size1, hidden_size2) # initialises weights and biases h12h2
                self.drop2 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h22h3 = nn.Linear(hidden_size2, hidden_size3) # initialises weights and biases h22h3
                self.drop3 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h32o = nn.Linear(hidden_size3, output_size) # initialises weights and biases h32o

            def forward(self, x):
                x = torch.relu(self.i2h1(x)) # relu activation @ hidden layer
                x = torch.relu(self.h12h2(x)) # relu activation @ hidden layer
                x = torch.relu(self.h22h3(x)) # relu activation @ hidden layer
                x = torch.sigmoid(self.h32o(x)) # sigmoid activation @ output layer
                return x

        num_of_input = VOCAB_SIZE # Bag word restriction has to be equal to vocabulary
        num_of_hidden1 = 30
        num_of_hidden2 = 20
        num_of_hidden3 = 10
        num_of_output = 1 # binary sentiment classes (+ve, -ve)

        bow = BOWClassifier(num_of_input, num_of_hidden1, num_of_hidden2, num_of_hidden3, num_of_output).to('cuda:0') # initialises weights and biases
        
        # define a loss function and 
        loss_function = nn.HingeEmbeddingLoss()
        
    # define an optimizer
    if(loss_fun == 'ce'):
        opt = torch.optim.SGD(bow.parameters(), lr = 0.0001)
    elif(loss_fun == 'mse'):
        opt = torch.optim.SGD(bow.parameters(), lr = 0.0003)
    elif(loss_fun == 'hinge'):
        opt = torch.optim.SGD(bow.parameters(), lr = 0.01)
    gc.collect()

    # Train The Model
    epochs = 100
    # the training loop
    total_time = 0.0
    prev_val_loss = float('inf')
    val_loss = 0
    early_stop_bow = 0

    for e in range(epochs):
        tic = time.time() # start the timer
        correct = 0
        cumulative_loss = 0
        incorrect = 0

        for i, instance in enumerate(train_data): # train_text_reviews 
            # get the training data
            label = train_text_labels[i] # get the label of the corresponding instace
            label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label

            bow.zero_grad() # reset the gradient for each instance

            pred = bow.forward(instance)

            loss = loss_function(pred, label) # compute the loss
            loss.backward() # backprop the loss
            opt.step() # performs parameter updation based on the current gradient

            cumulative_loss += float(loss.item()) # accumulate the loss over whole training sample

            pred_class = 1 if pred.item() > 0.5 else 0 # as sigmoid activated
            if(int(label) == pred_class): # counting correct prediction in each epoch
                correct += 1

        train_loss = float(cumulative_loss)/num_train_data
        train_accuracy = correct*float(100)/num_train_data

#         val_loss, val_accuracy, p, r, f = evaluate_validation_accuracy(val_data, bow) # test val-data-set on currently trained model
        val_loss = evaluate_validation_accuracy(val_data, bow) # test val-data-set on currently trained model

        if(prev_val_loss-val_loss < 0.0): # early stop if nearly no change
            bow = copy.deepcopy(early_stop_bow)
            break

        early_stop_bow = copy.deepcopy(bow)

        prev_val_loss = val_loss

        toc = time.time() # final time
        total_time += (toc-tic)

        print("Epoch {}/{}\n[On Training] ==> Time: {:.2f}s, Train Loss: {:.9f}, Train Accuracy: {:.2f}%".format(e+1, epochs, (toc-tic), train_loss, train_accuracy))
#         print("[On Validation] ==> Precision: {:.3f}, Recall: {:.3f}, F-Score: {:.3f}, Val loss: {:.9f}, Val Accuracy: {:.2f}% Total Time: {:.2f}s".format(p, r, f, val_loss, val_accuracy, total_time))
        print("[On Validation] ==> Val loss: {:.9f}, Total Time: {:.2f}s".format(val_loss, total_time))

    torch.save(bow, task_name)
    
    print('--- AFTER TRAINING ---\tModel: {}'.format(task_name))
    correct = 0
    tic = time.time()
    for i, instance in enumerate(data_test):
        label = test_text_labels[i] # get the label of the corresponding instace
        label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label

        pred = bow.forward(instance)
        pred_class = 1 if pred.item() > 0.5 else 0 # sigmoid activated

        if(int(label) == pred_class):
            correct += 1
    toc = time.time()
    print("[On Testing] Time: {}, Test Accuracy: {:.2f}%".format((toc-tic), correct*100/len(data_test)))

    print("\n")

Epoch 1/100
[On Training] ==> Time: 27.72s, Train Loss: 0.753298927, Train Accuracy: 49.80%
[On Validation] ==> Val loss: 0.490034675, Total Time: 27.72s


KeyboardInterrupt: ignored

In [0]:
# import matplotlib.pyplot as plt
# plt.figure("Image")
# plt.title("Loss vs Epoch")
# val_loss_history_plt =  [float(i)/sum(val_loss_history) for i in val_loss_history] # normalised between 0-1
# train_loss_history_plt =  [float(i)/sum(train_loss_history) for i in train_loss_history] # normalised between 0-1
# f_score_plt = [float(i)/sum(f_score) for i in f_score] # normalised between 0-1
# plt.plot(val_loss_history_plt, c="red", label="Validation Loss")
# plt.plot(train_loss_history_plt, c="blue", label = "Training Loss")
# plt.plot(f_score_plt, c="green", label = "F-Score")
# plt.legend()

In [0]:
from google.colab import files
temp_test = files.upload()

Saving Task3A.mdl to Task3A.mdl


In [0]:
# _model_ = torch.load(io.BytesIO(temp_test['Task2C.mdl']))
_model_ = torch.load('Task5C.mdl')

In [0]:
# for params in _model_.parameters():
#     print (params)

In [59]:
print('--- AFTER TRAINING ---')
correct = 0
tic = time.time()
for i, instance in enumerate(data_test):
    label = test_text_labels[i] # get the label of the corresponding instace
    label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label
    
    pred = _model_.forward(instance)
    pred_class = 1 if pred.item() > 0.5 else 0 # sigmoid activated
    
    if(int(label) == pred_class):
        correct += 1
toc = time.time()
print("Time: {}, Test Accuracy: {:.2f}%".format((toc-tic), correct*100/len(data_test)))

--- AFTER TRAINING ---
Time: 2.400538921356201, Test Accuracy: 87.58%


In [0]:
# predict on test dataset
# with open("Task1A.csv", "wb") as _f:
#     writer = csv.writer(_f)
#     writer.writerows(a)

In [0]:
np.save('vocab.npy', word_to_ix)

from google.colab import files
files.download("vocab.npy")